In [ ]:
# prompt: google drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
from google.colab import output

!pip install wandb -qU
output.clear()

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
import  pandas as  pd
import numpy as np
import os
from PIL import Image
import matplotlib.pyplot as plt
from torchvision.transforms.functional import to_pil_image
from torch.utils.data import random_split,DataLoader
from torchvision.transforms import functional as TF

Hyperparameter

In [ ]:
learning_rate = 0.001
epochs = 30
batch_size = 64

import data
DataLoader and split validation set

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/soowha/train.csv')
train['label'] = train['label'].replace({'10-1':0, '10-2':10})
train.to_csv('/content/drive/MyDrive/soowha/train.csv')

In [ ]:
class trainDataset(Dataset):
    def __init__(self , img_dir , label_csv , transform = None):

        self.img_dir = img_dir
        self.label_csv = pd.read_csv(label_csv)
        self.transform = transform

    def __len__(self):
        return min(300, len(self.label_csv))  # 최대 200개의 샘플만 반환합니다.

    def __getitem__(self , idx):
        img_name = os.path.join(self.img_dir ,str(self.label_csv.iloc[idx+1 , 0]).zfill(3)+'.png')
        image = Image.open(img_name)
        label = int(self.label_csv.iloc[idx , -1])

        if image.width > image.height:
            image = TF.rotate(image,-90)    # make the direction left to up


        if self.transform:
            image = self.transform(image)

        return image , label


#path
train_img_dir='/content/drive/MyDrive/soowha/224_train'
train_label_csv='/content/drive/MyDrive/soowha/train.csv'


transform = transforms.Compose([
    transforms.ToTensor()
])

train_data = trainDataset(train_img_dir, train_label_csv, transform = transform)

In [ ]:
# def calculate_mean_std(loader):
#     channels_sum, channels_squared_sum, num_batches = 0, 0, 0

#     for data, _ in loader: # data shape is [batchsize, channels, height, width]
#         batch = data.size(0)
#         data = data.view(batch, data.size(1), -1) # reshape to make h*w
#         channels_sum += torch.mean(data, dim=[0, 2]) * batch # get mean along channel multiply batch as the last batch is not 64
#         channels_squared_sum += torch.mean(data ** 2, dim=[0, 2]) * batch
#         num_batches += batch

#     mean = channels_sum / num_batches
#     std = (channels_squared_sum / num_batches - mean ** 2) ** 0.5

#     return mean, std

# train_loader = DataLoader(train_data, batch_size=64, shuffle=False)

# mean, std = calculate_mean_std(train_loader)
# print(f"Mean: {mean}")
# print(f"Std: {std}")

In [ ]:
transform_normalize= transforms.Compose([
    transforms.ToTensor(),
])

total_samples = len(train_data)
train_size = int(0.8 * total_samples)
valid_size = total_samples - train_size
train_data_split, valid_data_spilt = random_split(train_data, [train_size, valid_size])

Import and login to wandb

In [ ]:
import wandb
#login
wandb.login()


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
!pip install efficientnet_pytorch

  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached 

In [ ]:
import torch
import torch.nn as nn
from efficientnet_pytorch import EfficientNet

class EfficientNetModel(nn.Module):
    def __init__(self, num_classes=11, dropout_rate=0.0, use_bn=False):
        super(EfficientNetModel, self).__init__()
        self.effnet = EfficientNet.from_pretrained('efficientnet-b5')

        self.features = nn.Sequential(
            self.effnet,
            nn.Dropout(p=dropout_rate),
        )

        # BN 옵션
        if use_bn:
            self.features.add_module("batch_norm", nn.BatchNorm1d(self.effnet._fc.in_features))

        # BN 후의 피처 크기를 유지하면서 최종 분류를 위한 선형 레이어 추가
        self.fc = nn.Linear(self.effnet._fc.in_features, num_classes)

        # 기존 원형 레이어 제거
        self.effnet._fc = nn.Identity()

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)  # 배치 정규화를 위해 필요한 경우, 차원 조정
        x = self.fc(x)
        return x

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

modify to apply wandb


In [ ]:
import torch
import torchvision.transforms.functional as TF
from PIL import ImageDraw, ImageFont
from torchvision.transforms import ToPILImage
from sklearn.metrics import precision_score, recall_score, f1_score

import wandb
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

def metrics(true, pred):
    """계산된 메트릭을 반환"""
    precision = precision_score(true, pred, average='macro', zero_division=0)
    recall = recall_score(true, pred, average='macro', zero_division=0)
    f1 = f1_score(true, pred, average='macro', zero_division=0)
    return precision, recall, f1

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_path = './'

import tqdm

def train(model, train_loader, lossFunction, optimizer, device):
    model.train()
    train_loss = 0.0
    train_accuracy = 0.0
    all_preds = []
    all_labels = []

    for batch_idx, (image, y) in enumerate(train_loader):
        image = image.to(device)
        y = y.to(device)

        prediction = model(image)
        loss = lossFunction(prediction, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        preds = prediction.argmax(1)
        train_accuracy += (preds == y).type(torch.float).sum().item()

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(y.cpu().numpy())

    train_loss /= len(train_loader)
    train_accuracy /= len(train_loader.dataset)
    train_precision, train_recall, train_f1 = metrics(all_labels, all_preds)
    return train_loss, train_accuracy, train_precision, train_recall, train_f1

def evaluate(model, data_loader, device, lossFunction):
    model.eval()
    validation_loss = 0.0
    correct = 0.0
    total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for image, y in data_loader:
            image = image.to(device)
            y = y.to(device)

            prediction = model(image)
            validation_loss += lossFunction(prediction, y).item()

            preds = prediction.argmax(1)
            correct += (preds == y).sum().item()
            total += y.size(0)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y.cpu().numpy())

    validation_loss /= len(data_loader)
    validation_accuracy = correct / total
    validation_precision, validation_recall, validation_f1 = metrics(all_labels, all_preds)
    return validation_accuracy, validation_loss, validation_precision, validation_recall, validation_f1

def add_text_to_image(image, label, pred):
    """이미지에 레이블과 예측 값을 텍스트로 추가"""
    draw = ImageDraw.Draw(image)
    font = ImageFont.load_default()
    text = f"Label: {label}, Pred: {pred}"
    draw.text((10, 10), text, font=font, fill="white")
    return image

def run_sweep(config=None):
    with wandb.init(config=config):
        config=wandb.config
        train_loader = DataLoader(train_data_split, batch_size=config['batch_size'], shuffle = True)
        valid_loader = DataLoader(valid_data_spilt, batch_size=config['batch_size'], shuffle = False)


        model = EfficientNetModel(num_classes=11, dropout_rate=config.dropout, use_bn=config.use_bn)
        lossFunction = nn.CrossEntropyLoss()

       # 옵티마이저 선택
        if config.optimizer_type == 'adam':
            optimizer = optim.Adam(model.parameters(), lr=config.learning_rate, weight_decay=config.weight_decay, eps=config.eps)
        elif config.optimizer_type == 'adamw':
            optimizer = optim.AdamW(model.parameters(), lr=config.learning_rate, weight_decay=config.weight_decay, eps=config.eps)


        model.to(device)

        best_validation_accuracy = 0.0
        for epoch in range(epochs):
            print(f"Epoch {epoch + 1} of {epochs}")
            train_loss, train_accuracy, train_precision, train_recall, train_f1 = train(model, train_loader, lossFunction, optimizer, device)
            validation_accuracy, validation_loss, validation_precision, validation_recall, validation_f1 = evaluate(model, valid_loader, device, lossFunction)
            print(f"Accuracy on validation data: {validation_accuracy:.2f}%")

            sample_batch, labels = next(iter(valid_loader))
            predictions = model(sample_batch.to(device)).argmax(dim=1)
            images = [ToPILImage()(img) for img in sample_batch]

            logged_images = []
            for img, label, pred in zip(images, labels, predictions):
                annotated_image = add_text_to_image(img, label.item(), pred.item())
                logged_images.append(wandb.Image(annotated_image, caption=f"Label: {label}, Pred: {pred}"))

            wandb.log({
                "train_acc": train_accuracy,
                "train_loss": train_loss,
                "train_precision": train_precision,
                "train_recall": train_recall,
                "train_f1": train_f1,

                "validation_acc": validation_accuracy,
                "validation_loss": validation_loss,
                "validation_precision": validation_precision,
                "validation_recall": validation_recall,
                "validation_f1": validation_f1,
                "logged_images": logged_images
            })

            if validation_accuracy > best_validation_accuracy:
                best_validation_accuracy = validation_accuracy
                torch.save(model.state_dict(), f'{model_path}/best.pth')

        print("complete!")

In [ ]:
sweep_config = {
    'method': 'random'
    #'method' : 'bayes'
    }

metric = {
    'name': 'validation_acc',
    'goal': 'maximize'
    }

parameters_dict = {
    'optimizer_type': {
        'values': ['adam', 'adamw']
        },
    'batch_size': {
        'values': [8, 16, 32]
        },
    'learning_rate': {
        'min': 0.005,
        'max': 0.1,
      },
    'dropout' : {
        'values' : [0.0, 0.1, 0.2, 0.3, 0.4]
        },
    'weight_decay': {
        'min': 0.001,
        'max': 0.01,
        },
    'eps': {
        'values': [1e-09*5,1e-08]
        },
    'use_bn' : {
        'values': [True, False]
    }
}

sweep_config['metric'] = metric
sweep_config['parameters'] = parameters_dict

In [ ]:
sweep_id = wandb.sweep(sweep_config,entity='joon_dsjoh24', project="Interactive AI")

wandb.agent(sweep_id, run_sweep, count=10)

Create sweep with ID: 97ww495w
Sweep URL: https://wandb.ai/joon_dsjoh24/Interactive%20AI/sweeps/97ww495w


wandb: Agent Starting Run: 7umn4p8s with config:
wandb: 	batch_size: 16
wandb: 	dropout: 0.1
wandb: 	eps: 5e-09
wandb: 	learning_rate: 0.006643035422800789
wandb: 	optimizer_type: adamw
wandb: 	use_bn: True
wandb: 	weight_decay: 0.00402298146010238
wandb: Currently logged in as: dsjoh24 (joon_dsjoh24). Use `wandb login --relogin` to force relogin


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b5-b6417697.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b5-b6417697.pth
100%|██████████| 117M/117M [00:00<00:00, 201MB/s] 


Loaded pretrained weights for efficientnet-b5
Epoch 1 of 30
Accuracy on validation data: 0.08%
Epoch 2 of 30
Accuracy on validation data: 0.18%
Epoch 3 of 30
Accuracy on validation data: 0.15%
Epoch 4 of 30
Accuracy on validation data: 0.08%
Epoch 5 of 30
Accuracy on validation data: 0.08%
Epoch 6 of 30
Accuracy on validation data: 0.08%
Epoch 7 of 30
Accuracy on validation data: 0.08%
Epoch 8 of 30
Accuracy on validation data: 0.02%
Epoch 9 of 30
Accuracy on validation data: 0.10%
Epoch 10 of 30
Accuracy on validation data: 0.08%
Epoch 11 of 30
Accuracy on validation data: 0.02%
Epoch 12 of 30
Accuracy on validation data: 0.08%
Epoch 13 of 30
Accuracy on validation data: 0.13%
Epoch 14 of 30
Accuracy on validation data: 0.10%
Epoch 15 of 30
Accuracy on validation data: 0.07%
Epoch 16 of 30
Accuracy on validation data: 0.00%
Epoch 17 of 30
Accuracy on validation data: 0.12%
Epoch 18 of 30
Accuracy on validation data: 0.12%
Epoch 19 of 30
Accuracy on validation data: 0.03%
Epoch 20 of 3

train_acc,▆▃▆▄▁▄▄▂▄▄▆█▃▄▄▂▃▃▆▆▆▅█▃▅▄▄▅▇▆
train_f1,▆▃▆▄▁▄▃▂▃▄▆▇▃▃▄▂▃▂▆▅▅▅▇▂▅▃▄▄█▅
train_loss,▃▂▃▃▂▅▅▅▄▄▄▃██▆█▇▅▆▅▆▄▃▃▄▄▃▄▁▄
train_precision,▅▃▅▄▁▃▂▁▃▃▄▆▂▂▃▁▂▁▅▄▄▄▆▂▄▃▃▃█▄
train_recall,▇▃▇▄▁▅▄▂▄▄▇█▃▄▄▂▃▃▆▆▆▆█▂▅▄▄▅█▆
validation_acc,▄█▇▄▄▄▄▂▅▄▂▄▆▅▄▁▅▅▂▄▇▂▂▂█▄▇▂▄▂
validation_f1,▂▄▃▂▂▂▂▁▄▅▁▃█▃▆▁▅▅▃▂▃▂▁▁▄▂▃▁▂▁
validation_loss,█▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
validation_precision,▁▂▂▁▁▁▁▁▇▃▁▂▄▂▃▁▃█▃▂▂▁▁▁▂▁▂▁▁▁
validation_recall,▅▅▅▅▅▅▅▅▅▆▂▅▆▅█▁▅▄▅▅▅▆▅▅▅▅▅▅▅▅
train_acc,0.15


...

Loaded pretrained weights for efficientnet-b5
Epoch 1 of 30
Accuracy on validation data: 0.15%
Epoch 2 of 30
Accuracy on validation data: 0.00%
Epoch 3 of 30
Accuracy on validation data: 0.00%
Epoch 4 of 30
Accuracy on validation data: 0.10%
Epoch 5 of 30
Accuracy on validation data: 0.02%
Epoch 6 of 30
Accuracy on validation data: 0.07%
Epoch 7 of 30
Accuracy on validation data: 0.02%
Epoch 8 of 30
Accuracy on validation data: 0.07%
Epoch 9 of 30
Accuracy on validation data: 0.15%
Epoch 10 of 30
Accuracy on validation data: 0.18%
Epoch 11 of 30
Accuracy on validation data: 0.15%
Epoch 12 of 30
Accuracy on validation data: 0.15%
Epoch 13 of 30
Accuracy on validation data: 0.05%
Epoch 14 of 30
Accuracy on validation data: 0.08%
Epoch 15 of 30
Accuracy on validation data: 0.10%
Epoch 16 of 30
Accuracy on validation data: 0.05%
Epoch 17 of 30
Accuracy on validation data: 0.15%
Epoch 18 of 30
Accuracy on validation data: 0.15%
Epoch 19 of 30
Accuracy on validation data: 0.15%
Epoch 20 of 3

train_acc,▃▅▁▃▇▇▂▅▃▂▅▅▁▃▄▂▆▂▄▄▃█▂▄▅▃▄▅▄▆
train_f1,▄▅▃▅█▇▄▇▃▃▆▄▂▃▄▁▅▃▄▅▂▇▃▅▇▄▃▅▅▆
train_loss,█▄▅▄▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▂▁▂▂▂
train_precision,▄▅▃▅█▇▄▇▃▃▆▅▂▃▅▁▇▄▅▆▂▆▄▅▇▄▃▅▆▆
train_recall,▄▅▂▄██▃▆▃▂▆▄▁▃▄▁▆▂▄▄▂█▂▅▇▄▄▅▄▆
validation_acc,▇▁▁▅▂▄▂▄▇█▇▇▃▄▅▃▇▇▇▁▄▄▂█▄▄▄▃▅▇
validation_f1,▇▁▁▅▂▄▂▄▇█▇▇▃▄▅▃▇▇▇▁▄▄▂█▄▄▄▃▅▇
validation_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
validation_precision,▇▁▁▅▂▄▂▄▇█▇▇▃▄▅▃▇▇▇▁▄▄▂█▄▄▄▃▅▇
validation_recall,█▁▁████████████████▁██████████
train_acc,0.10833


wandb: Agent Starting Run: pdccj6os with config:
wandb: 	batch_size: 8
wandb: 	dropout: 0.3
wandb: 	eps: 1e-08
wandb: 	learning_rate: 0.02704869322396635
wandb: 	optimizer_type: adamw
wandb: 	use_bn: True
wandb: 	weight_decay: 0.0028926811463568097


Loaded pretrained weights for efficientnet-b5
Epoch 1 of 30
Accuracy on validation data: 0.02%
Epoch 2 of 30
Accuracy on validation data: 0.05%
Epoch 3 of 30
Accuracy on validation data: 0.08%
Epoch 4 of 30
Accuracy on validation data: 0.22%
Epoch 5 of 30
Accuracy on validation data: 0.07%
Epoch 6 of 30
Accuracy on validation data: 0.05%
Epoch 7 of 30
Accuracy on validation data: 0.02%
Epoch 8 of 30
Accuracy on validation data: 0.02%
Epoch 9 of 30
Accuracy on validation data: 0.08%
Epoch 10 of 30
Accuracy on validation data: 0.22%
Epoch 11 of 30
Accuracy on validation data: 0.10%
Epoch 12 of 30
Accuracy on validation data: 0.05%
Epoch 13 of 30
Accuracy on validation data: 0.03%
Epoch 14 of 30
Accuracy on validation data: 0.02%
Epoch 15 of 30
Accuracy on validation data: 0.13%
Epoch 16 of 30
Accuracy on validation data: 0.08%
Epoch 17 of 30
Accuracy on validation data: 0.10%
Epoch 18 of 30
Accuracy on validation data: 0.12%
Epoch 19 of 30
Accuracy on validation data: 0.08%
Epoch 20 of 3

train_acc,▇█▅▄▆▃▄▄▁▇▄▄█▁▄▄▂▄▂▂▁▆▂▃▅▃▂▃▂▁
train_f1,▇█▆▃▆▃▄▄▂█▄▅▇▂▄▅▃▃▃▃▁▇▃▂▆▄▃▂▃▂
train_loss,▅▆████▇▆▆▅▄▄▃▂▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
train_precision,██▆▃▇▃▄▄▂█▄▅▇▂▄▆▃▃▃▃▁█▃▂▆▄▃▂▄▂
train_recall,▇█▅▄▆▃▄▄▂█▄▄▇▂▄▅▃▃▃▃▁▇▃▂▆▃▂▂▃▂
validation_acc,▁▂▃█▃▂▁▁▃█▄▂▂▁▅▃▄▄▃▂▃▂▅▃▂▆▂▂▁▄
validation_f1,▁▂▂▅▂▂▁▁▂█▅▃▂▁▅▃▄▅▃▂▂▂▅▄▂▅▃▂▁▃
validation_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
validation_precision,▁▂▂▄▂▂▁▁▂█▅▃▂▁▅▃▄▄▃▂▂▂▅▅▂▅▄▂▁▂
validation_recall,▄▄▄▄▄▄▄▄▄▅█▅▃▂▄▄▅▅▂▄▄▁▅▃▂▄▁▄▄▄
train_acc,0.07083


In [ ]:
sweep_config = {
    'method' : 'bayes'
    #'method' : 'grid'
    }

metric = {
    'name': 'validation_acc',
    'goal': 'maximize'
    }

parameters_dict = {
    'optimizer_type': {
        'values': ['adam', 'adamw']
        },
    'batch_size': {
        'values': [8, 16, 32]
        },
    'learning_rate': {
        'min': 0.005,
        'max': 0.1,
      },
    'dropout' : {
        'values' : [0.0, 0.1, 0.2, 0.3, 0.4]
        },
    'weight_decay': {
        'min': 0.001,
        'max': 0.01,
        },
    'eps': {
        'values': [1e-09*5,1e-08]
        },
    'use_bn' : {
        'values': [True, False]
    }
}

sweep_config['metric'] = metric
sweep_config['parameters'] = parameters_dict

In [ ]:
sweep_id = wandb.sweep(sweep_config,entity='joon_dsjoh24', project="Interactive AI")

wandb.agent(sweep_id, run_sweep, count=10)

Create sweep with ID: ln2re7kw
Sweep URL: https://wandb.ai/joon_dsjoh24/Interactive%20AI/sweeps/ln2re7kw


wandb: Agent Starting Run: rlndc2nr with config:
wandb: 	batch_size: 8
wandb: 	dropout: 0.3
wandb: 	eps: 5e-09
wandb: 	learning_rate: 0.028201557022945738
wandb: 	optimizer_type: adamw
wandb: 	use_bn: True
wandb: 	weight_decay: 0.001137070376727401


Loaded pretrained weights for efficientnet-b5
Epoch 1 of 30
Accuracy on validation data: 0.05%
Epoch 2 of 30
Accuracy on validation data: 0.08%
Epoch 3 of 30
Accuracy on validation data: 0.02%
Epoch 4 of 30
Accuracy on validation data: 0.22%
Epoch 5 of 30
Accuracy on validation data: 0.07%
Epoch 6 of 30
Accuracy on validation data: 0.07%
Epoch 7 of 30
Accuracy on validation data: 0.07%
Epoch 8 of 30
Accuracy on validation data: 0.08%
Epoch 9 of 30
Accuracy on validation data: 0.00%
Epoch 10 of 30
Accuracy on validation data: 0.08%
Epoch 11 of 30
Accuracy on validation data: 0.18%
Epoch 12 of 30
Accuracy on validation data: 0.08%
Epoch 13 of 30
Accuracy on validation data: 0.03%
Epoch 14 of 30
Accuracy on validation data: 0.05%
Epoch 15 of 30
Accuracy on validation data: 0.05%
Epoch 16 of 30
Accuracy on validation data: 0.00%
Epoch 17 of 30
Accuracy on validation data: 0.00%
Epoch 18 of 30
Accuracy on validation data: 0.02%
Epoch 19 of 30
Accuracy on validation data: 0.02%
Epoch 20 of 3

train_acc,▃▄▅▅▃▇▅▄▄▄▅▇▅▄▂▄▂█▃▄▄▅▁▇▅▃▂▃▆▇
train_f1,▂▄▄▄▂▆▄▄▃▃▄▆▃▃▁▄▂█▂▃▃▄▁▆▅▂▁▁▅▆
train_loss,▅▇███▇▆▇▅▅▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
train_precision,▂▄▄▄▂▆▄▄▃▄▄▆▃▃▁▄▂█▂▃▃▄▂▆▅▂▂▁▅▆
train_recall,▂▄▄▄▂▆▄▄▃▄▄▆▃▃▁▄▂█▂▃▃▄▁▆▅▂▁▂▅▆
validation_acc,▃▄▂█▃▃▃▄▁▄▇▄▂▃▃▁▁▂▂▆▄▂▆▂▆▇▇▃▂▅
validation_f1,▂▃▁▆▃▃▃▃▁▃▅▃▃▃▄▁▁▁▁▄▃▁▄▁▄▅█▃▂▆
validation_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
validation_precision,▂▂▁▄▂▂▂▂▁▂▃▂▂▂▅▁▁▁▁▃▂▁▃▁▃▃█▂▁▅
validation_recall,▇▇▇▇▇▇▇▇▁▇▇▇▂▂▄▁▁▇▇▇▇▇▇▇▇▇█▆▃▆
train_acc,0.11667


wandb: Agent Starting Run: l5clau5u with config:
wandb: 	batch_size: 8
wandb: 	dropout: 0
wandb: 	eps: 1e-08
wandb: 	learning_rate: 0.05749489781064298
wandb: 	optimizer_type: adam
wandb: 	use_bn: False
wandb: 	weight_decay: 0.005576519112642351


...

Loaded pretrained weights for efficientnet-b5
Epoch 1 of 30
Accuracy on validation data: 0.22%
Epoch 2 of 30
Accuracy on validation data: 0.00%
Epoch 3 of 30
Accuracy on validation data: 0.08%
Epoch 4 of 30
Accuracy on validation data: 0.00%
Epoch 5 of 30
Accuracy on validation data: 0.05%
Epoch 6 of 30
Accuracy on validation data: 0.22%
Epoch 7 of 30
Accuracy on validation data: 0.05%
Epoch 8 of 30
Accuracy on validation data: 0.05%
Epoch 9 of 30
Accuracy on validation data: 0.10%
Epoch 10 of 30
Accuracy on validation data: 0.12%
Epoch 11 of 30
Accuracy on validation data: 0.05%
Epoch 12 of 30
Accuracy on validation data: 0.02%
Epoch 13 of 30
Accuracy on validation data: 0.00%
Epoch 14 of 30
Accuracy on validation data: 0.08%
Epoch 15 of 30
Accuracy on validation data: 0.07%
Epoch 16 of 30
Accuracy on validation data: 0.08%
Epoch 17 of 30
Accuracy on validation data: 0.02%
Epoch 18 of 30
Accuracy on validation data: 0.07%
Epoch 19 of 30
Accuracy on validation data: 0.00%
Epoch 20 of 3

train_acc,▅▃▃▃▃▄▃▅▅▁▂▆▂▃▂▅▅▄▆▅▇█▄▃▅▅▇▅▅▇
train_f1,▅▃▃▃▃▅▃▆▆▁▂▆▃▃▂▅▆▄▇▇██▃▃▆▅▇▆▅█
train_loss,█▅▂▂▂▃▂▂▃▂▂▁▂▁▁▂▁▁▁▁▁▁▁▂▂▁▂▃▂▂
train_precision,▅▃▂▃▃▄▃▅▆▁▂▆▂▃▂▅▅▃▆▆▇▇▃▃▆▄▆▆▅█
train_recall,▅▃▃▃▃▄▃▆▆▁▂▆▂▃▂▅▆▄▆▆▇█▃▃▅▅▇▆▅▇
validation_acc,█▁▄▁▃█▃▃▄▅▃▂▁▄▃▄▂▃▁▄▇▇▄▁▄▃▃▇▃▃
validation_f1,▆▁▃▁▂▆▂▂▃▅▂▁▁▃▃▃▁▃▁▃▅▅█▁▄▂▂▅▂▃
validation_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
validation_precision,▄▁▂▁▂▄▂▂▂▄▂▁▁▂▂▂▁▂▁▂▃▃█▁▂▂▂▃▂▂
validation_recall,▇▁▇▁▇▇▇▇▇█▇▇▁▇▇▇▇▃▁▇▇▇█▁▇▇▇▇▇▇
train_acc,0.14167


wandb: Agent Starting Run: 46u06la3 with config:
wandb: 	batch_size: 8
wandb: 	dropout: 0
wandb: 	eps: 5e-09
wandb: 	learning_rate: 0.05970276495842102
wandb: 	optimizer_type: adam
wandb: 	use_bn: False
wandb: 	weight_decay: 0.009197054286159582


Loaded pretrained weights for efficientnet-b5
Epoch 1 of 30
Accuracy on validation data: 0.22%
Epoch 2 of 30
Accuracy on validation data: 0.05%
Epoch 3 of 30
Accuracy on validation data: 0.02%
Epoch 4 of 30
Accuracy on validation data: 0.18%
Epoch 5 of 30
Accuracy on validation data: 0.18%
Epoch 6 of 30
Accuracy on validation data: 0.22%
Epoch 7 of 30
Accuracy on validation data: 0.02%
Epoch 8 of 30
Accuracy on validation data: 0.15%
Epoch 9 of 30
Accuracy on validation data: 0.08%
Epoch 10 of 30
Accuracy on validation data: 0.07%
Epoch 11 of 30
Accuracy on validation data: 0.05%
Epoch 12 of 30
Accuracy on validation data: 0.10%
Epoch 13 of 30
Accuracy on validation data: 0.18%
Epoch 14 of 30
Accuracy on validation data: 0.10%
Epoch 15 of 30
Accuracy on validation data: 0.08%
Epoch 16 of 30
Accuracy on validation data: 0.18%
Epoch 17 of 30
Accuracy on validation data: 0.08%
Epoch 18 of 30
Accuracy on validation data: 0.18%
Epoch 19 of 30
Accuracy on validation data: 0.02%
Epoch 20 of 3

train_acc,▄▄▄▆▆▆▃▆▄▄▅▃▃▃▂▁▃▂▅▃▅▄▅█▅▃▅▅▄▅
train_f1,▅▄▃▆▄▄▂▃▂▃▃▂▁▂▁▁▂▁▄▃▄▃▃█▅▂▃▅▃▅
train_loss,█▄▃▂▁▁▁▁▁▁▁▁▁▁▂▂▁▁▁▂▂▂▂▂▂▂▂▂▂▂
train_precision,▆▅▃▆▄▄▂▄▂▃▃▁▁▂▂▂▃▁▅▄▆▃▃█▅▂▃▆▂▆
train_recall,▅▄▃▆▅▅▂▄▃▃▄▃▂▂▁▁▂▂▄▃▄▃▃█▄▂▃▅▃▄
validation_acc,█▃▂▇▇█▂▆▄▃▃▄▇▄▄▇▄▇▂▂▂▃▁▄▃▄▃▆▃▆
validation_f1,█▃▂▇▇█▂▆▄▃▃▅▇▅▄▇▄▇▂▂▂▃▁▄▃▅▃▆▃▆
validation_loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▃▃▃▂▃▂▂▂▁
validation_precision,█▃▂▇▇█▂▆▄▃▃▄▇▄▄▇▄▇▂▂▂▃▁▄▃▄▃▆▃▆
validation_recall,██████████████████████▁███████
train_acc,0.10417


In [ ]:
import torch
import torchvision.transforms.functional as TF
from PIL import ImageDraw, ImageFont
from torchvision.transforms import ToPILImage
from sklearn.metrics import precision_score, recall_score, f1_score

import wandb
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

def metrics(true, pred):
    """계산된 메트릭을 반환"""
    precision = precision_score(true, pred, average='macro', zero_division=0)
    recall = recall_score(true, pred, average='macro', zero_division=0)
    f1 = f1_score(true, pred, average='macro', zero_division=0)
    return precision, recall, f1

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_path = './'

import tqdm

def train(model, train_loader, lossFunction, optimizer, device):
    model.train()
    train_loss = 0.0
    train_accuracy = 0.0
    all_preds = []
    all_labels = []

    for batch_idx, (image, y) in enumerate(train_loader):
        image = image.to(device)
        y = y.to(device)

        prediction = model(image)
        loss = lossFunction(prediction, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        preds = prediction.argmax(1)
        train_accuracy += (preds == y).type(torch.float).sum().item()

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(y.cpu().numpy())

    train_loss /= len(train_loader)
    train_accuracy /= len(train_loader.dataset)
    train_precision, train_recall, train_f1 = metrics(all_labels, all_preds)
    return train_loss, train_accuracy, train_precision, train_recall, train_f1

def evaluate(model, data_loader, device, lossFunction):
    model.eval()
    validation_loss = 0.0
    correct = 0.0
    total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for image, y in data_loader:
            image = image.to(device)
            y = y.to(device)

            prediction = model(image)
            validation_loss += lossFunction(prediction, y).item()

            preds = prediction.argmax(1)
            correct += (preds == y).sum().item()
            total += y.size(0)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y.cpu().numpy())

    validation_loss /= len(data_loader)
    validation_accuracy = correct / total
    validation_precision, validation_recall, validation_f1 = metrics(all_labels, all_preds)
    return validation_accuracy, validation_loss, validation_precision, validation_recall, validation_f1

def add_text_to_image(image, label, pred):
    """이미지에 레이블과 예측 값을 텍스트로 추가"""
    draw = ImageDraw.Draw(image)
    font = ImageFont.load_default()
    text = f"Label: {label}, Pred: {pred}"
    draw.text((10, 10), text, font=font, fill="white")
    return image

def run_sweep(config=None):
    with wandb.init(config=config):
        config=wandb.config
        train_loader = DataLoader(train_data_split, batch_size=config['batch_size'], shuffle = True)
        valid_loader = DataLoader(valid_data_spilt, batch_size=config['batch_size'], shuffle = False)

        seed_everything(config.seed) # Seed 고정

        model = EfficientNetModel(num_classes=11, dropout_rate=config.dropout, use_bn=config.use_bn)
        lossFunction = nn.CrossEntropyLoss()

       # 옵티마이저 선택
        if config.optimizer_type == 'adam':
            optimizer = optim.Adam(model.parameters(), lr=config.learning_rate, weight_decay=config.weight_decay, eps=config.eps)
        elif config.optimizer_type == 'adamw':
            optimizer = optim.AdamW(model.parameters(), lr=config.learning_rate, weight_decay=config.weight_decay, eps=config.eps)


        model.to(device)

        best_validation_accuracy = 0.0
        for epoch in range(epochs):
            print(f"Epoch {epoch + 1} of {epochs}")
            train_loss, train_accuracy, train_precision, train_recall, train_f1 = train(model, train_loader, lossFunction, optimizer, device)
            validation_accuracy, validation_loss, validation_precision, validation_recall, validation_f1 = evaluate(model, valid_loader, device, lossFunction)
            print(f"Accuracy on validation data: {validation_accuracy:.2f}%")

            sample_batch, labels = next(iter(valid_loader))
            predictions = model(sample_batch.to(device)).argmax(dim=1)
            images = [ToPILImage()(img) for img in sample_batch]

            logged_images = []
            for img, label, pred in zip(images, labels, predictions):
                annotated_image = add_text_to_image(img, label.item(), pred.item())
                logged_images.append(wandb.Image(annotated_image, caption=f"Label: {label}, Pred: {pred}"))

            wandb.log({
                "train_acc": train_accuracy,
                "train_loss": train_loss,
                "train_precision": train_precision,
                "train_recall": train_recall,
                "train_f1": train_f1,

                "validation_acc": validation_accuracy,
                "validation_loss": validation_loss,
                "validation_precision": validation_precision,
                "validation_recall": validation_recall,
                "validation_f1": validation_f1,
                "logged_images": logged_images
            })

            if validation_accuracy > best_validation_accuracy:
                best_validation_accuracy = validation_accuracy
                torch.save(model.state_dict(), f'{model_path}/best.pth')

        print("complete!")

In [ ]:
import random

sweep_config = {
    'method': 'random'
    }

metric = {
    'name': 'validation_acc',
    'goal': 'maximize'
    }

parameters_dict = {
    'optimizer_type': {
        'values': ['adamw']
        },
    'batch_size': {
        'values': [16]
        },
    'learning_rate': {
        'values': [0.006217484671852791]
      },
    'dropout' : {
        'values' : [0.1]
        },
    'weight_decay': {
        'values': [0.005484089961555269]
        },
    'eps': {
        'values': [0.00000001]
        },
    'use_bn' : {
        'values': [False]
    },
    'seed' : {
        'distribution': 'int_uniform',
        'max': 3000,
        'min': 1
    }
}

sweep_config['metric'] = metric
sweep_config['parameters'] = parameters_dict

In [ ]:
sweep_id = wandb.sweep(sweep_config,entity='joon_dsjoh24', project="Interactive AI")

wandb.agent(sweep_id, run_sweep, count=10)

Create sweep with ID: ylkalzn0
Sweep URL: https://wandb.ai/joon_dsjoh24/Interactive%20AI/sweeps/ylkalzn0


wandb: Agent Starting Run: bzccmsfq with config:
wandb: 	batch_size: 16
wandb: 	dropout: 0.1
wandb: 	eps: 1e-08
wandb: 	learning_rate: 0.006217484671852791
wandb: 	optimizer_type: adamw
wandb: 	seed: 765
wandb: 	use_bn: False
wandb: 	weight_decay: 0.005484089961555269


Loaded pretrained weights for efficientnet-b5
Epoch 1 of 30
Accuracy on validation data: 0.00%
Epoch 2 of 30
Accuracy on validation data: 0.10%
Epoch 3 of 30
Accuracy on validation data: 0.08%
Epoch 4 of 30
Accuracy on validation data: 0.08%
Epoch 5 of 30
Accuracy on validation data: 0.08%
Epoch 6 of 30
Accuracy on validation data: 0.08%
Epoch 7 of 30
Accuracy on validation data: 0.12%
Epoch 8 of 30
Accuracy on validation data: 0.17%
Epoch 9 of 30
Accuracy on validation data: 0.00%
Epoch 10 of 30
Accuracy on validation data: 0.15%
Epoch 11 of 30
Accuracy on validation data: 0.15%
Epoch 12 of 30
Accuracy on validation data: 0.23%
Epoch 13 of 30
Accuracy on validation data: 0.12%
Epoch 14 of 30
Accuracy on validation data: 0.22%
Epoch 15 of 30
Accuracy on validation data: 0.25%
Epoch 16 of 30
Accuracy on validation data: 0.28%
Epoch 17 of 30
Accuracy on validation data: 0.02%
Epoch 18 of 30
Accuracy on validation data: 0.13%
Epoch 19 of 30
Accuracy on validation data: 0.20%
Epoch 20 of 3

train_acc,▂▁▂▂▂▃▃▃▄▄▄▄▅▄▅▆▅▆▆▆▇▆▇▇▇▇████
train_f1,▂▁▂▂▂▃▂▃▃▄▄▄▅▄▅▆▅▅▆▆▇▆▇▇▇▇████
train_loss,██▇▇▆▆▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▃▂▂▂▂▁▁▁▁
train_precision,▂▁▂▂▂▃▃▃▃▄▄▄▅▅▅▆▆▅▆▆▇▆▇▇▇▇████
train_recall,▂▁▂▂▂▃▃▃▃▄▄▄▅▄▅▆▅▆▆▆▇▆▇▇▇▇████
validation_acc,▁▃▂▂▂▂▃▄▁▃▃▅▃▄▅▅▁▃▄▃▄▅█▅▄▅▅▆▇▆
validation_f1,▁▁▁▂▁▂▂▂▁▂▂▃▃▂▃▄▁▂▄▃▄▅█▄▃▄▄▆█▇
validation_loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
validation_precision,▁▁▁▁▁▁▂▂▁▂▂▂▃▁▂▄▁▃▃▃▃▄▅▄▃▃▄▅█▅
validation_recall,▁▃▃▃▃▃▃▃▁▃▃▄▄▃▄▄▃▃▅▃▆▇█▄▄▄▄▅▇█
train_acc,0.8875


wandb: Agent Starting Run: x1hhlc8j with config:
wandb: 	batch_size: 16
wandb: 	dropout: 0.1
wandb: 	eps: 1e-08
wandb: 	learning_rate: 0.006217484671852791
wandb: 	optimizer_type: adamw
wandb: 	seed: 2732
wandb: 	use_bn: False
wandb: 	weight_decay: 0.005484089961555269


...

Loaded pretrained weights for efficientnet-b5
Epoch 1 of 30
Accuracy on validation data: 0.08%
Epoch 2 of 30
Accuracy on validation data: 0.22%
Epoch 3 of 30
Accuracy on validation data: 0.10%
Epoch 4 of 30
Accuracy on validation data: 0.18%
Epoch 5 of 30
Accuracy on validation data: 0.10%
Epoch 6 of 30
Accuracy on validation data: 0.07%
Epoch 7 of 30
Accuracy on validation data: 0.22%
Epoch 8 of 30
Accuracy on validation data: 0.22%
Epoch 9 of 30
Accuracy on validation data: 0.08%
Epoch 10 of 30
Accuracy on validation data: 0.08%
Epoch 11 of 30
Accuracy on validation data: 0.10%
Epoch 12 of 30
Accuracy on validation data: 0.07%
Epoch 13 of 30
Accuracy on validation data: 0.12%
Epoch 14 of 30
Accuracy on validation data: 0.13%
Epoch 15 of 30
Accuracy on validation data: 0.22%
Epoch 16 of 30
Accuracy on validation data: 0.15%
Epoch 17 of 30
Accuracy on validation data: 0.25%
Epoch 18 of 30
Accuracy on validation data: 0.10%
Epoch 19 of 30
Accuracy on validation data: 0.22%
Epoch 20 of 3

train_acc,▁▁▁▂▂▃▄▃▄▄▄▅▅▆▆▆▆▆▆▇▇▇▇▇▇█████
train_f1,▁▁▁▁▂▃▃▃▄▄▄▅▅▆▆▆▆▆▆▇▇▇▇▇▇█████
train_loss,██▇▇▆▅▅▅▄▄▄▃▃▃▂▂▂▃▃▂▂▁▁▂▁▁▁▁▁▁
train_precision,▁▁▁▁▂▃▃▄▄▄▄▅▅▆▆▆▆▆▆▇▇▇▇▇▇█████
train_recall,▁▁▁▂▂▃▃▃▄▄▄▅▅▆▆▆▆▆▆▇▇▇▇▇▇█████
validation_acc,▁▃▁▃▁▁▃▃▁▁▁▁▂▂▃▂▄▁▃▅▃▄▆▄▆▇█▆▄▅
validation_f1,▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▃▂▄▄▃▃▆▄▆▅█▅▄▆
validation_loss,▁▅█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
validation_precision,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▃▁▄▃▄▃▅▅█▅▇▆▄▇
validation_recall,▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▃▃▃▄▃▃▆▄▆▅█▅▆▅
train_acc,0.92083


wandb: Agent Starting Run: nfdiahfd with config:
wandb: 	batch_size: 16
wandb: 	dropout: 0.1
wandb: 	eps: 1e-08
wandb: 	learning_rate: 0.006217484671852791
wandb: 	optimizer_type: adamw
wandb: 	seed: 2175
wandb: 	use_bn: False
wandb: 	weight_decay: 0.005484089961555269


Loaded pretrained weights for efficientnet-b5
Epoch 1 of 30
Accuracy on validation data: 0.00%
Epoch 2 of 30
Accuracy on validation data: 0.08%
Epoch 3 of 30
Accuracy on validation data: 0.18%
Epoch 4 of 30
Accuracy on validation data: 0.18%
Epoch 5 of 30
Accuracy on validation data: 0.22%
Epoch 6 of 30
Accuracy on validation data: 0.08%
Epoch 7 of 30
Accuracy on validation data: 0.02%
Epoch 8 of 30
Accuracy on validation data: 0.17%
Epoch 9 of 30
Accuracy on validation data: 0.22%
Epoch 10 of 30
Accuracy on validation data: 0.22%
Epoch 11 of 30
Accuracy on validation data: 0.10%
Epoch 12 of 30
Accuracy on validation data: 0.13%
Epoch 13 of 30
Accuracy on validation data: 0.22%
Epoch 14 of 30
Accuracy on validation data: 0.18%
Epoch 15 of 30
Accuracy on validation data: 0.12%
Epoch 16 of 30
Accuracy on validation data: 0.18%
Epoch 17 of 30
Accuracy on validation data: 0.15%
Epoch 18 of 30
Accuracy on validation data: 0.18%
Epoch 19 of 30
Accuracy on validation data: 0.15%
Epoch 20 of 3

train_acc,▁▁▁▁▂▂▃▄▃▅▅▅▆▆▅▅▆▇▆▆▆▇▇▇▇▇████
train_f1,▁▁▁▁▂▂▂▃▃▄▅▄▅▅▅▅▆▇▆▆▆▇▇▇▇▇█▇██
train_loss,██▇▇▆▆▅▅▅▄▃▄▃▃▃▃▂▂▃▃▂▂▂▂▁▂▁▁▁▁
train_precision,▁▁▁▂▂▂▂▃▃▄▅▄▅▅▅▅▆▇▆▆▆▇▇▇▇▇████
train_recall,▁▁▁▁▂▂▃▃▃▄▅▄▅▅▅▅▆▇▆▆▆▇▇▇▇▇█▇██
validation_acc,▁▂▃▃▄▂▁▃▄▄▂▃▄▃▂▃▃▃▃▄▄▃▅▆▆▆▅█▆█
validation_f1,▁▁▂▂▂▂▁▂▂▂▁▁▂▃▂▃▂▂▂▃▂▂▃▄▅▆▅▇▆█
validation_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
validation_precision,▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▁▃▁▃▂▂▄▄▄▅▅▅▆█
validation_recall,▁▃▃▃▃▂▃▃▃▃▃▃▃▅▃▅▃▅▃▄▄▃▄▆▇▇▆█▇█
train_acc,0.925
